## Alpha-Go

## Introduction
The board is treated as a 19x19 image and use convolutional layers to construct a
representation of the position. The approach is like a PI: evaluating positions
using a value network, and sampling actions using a policy network.

The procedures follow:
1. A fast rollout policy $p_{\pi}$ and supervised learning policy network $p_{\sigma}$ are trained to predict human expert moves in a dataset of positions.
2. A RL policy network $\pi_{\rho}$ is initialized to $p_{\sigma}$ and is then improved by policy gradient learning to maximize the outcome against previous versions of the policy network.
3. A new dataset is generated by playing games of self-play with the RL policy network.
4. A value network $v_{\theta}$ is trained by regression to predict the expected outcome.

The SL policy network $p_{\sigma} (a | s)$ alternates between convolutional layers with weights $\sigma$, and RELU. A final softmax layer outputs a probability distribution over all legal moves a. The input s to the policy network is a simple representation of the board state. The policy network is trained on randomly sampled state-action pairs, using stochastic gradient ascent to maximize the likelihood of the human move a selected in state s. Rollout policy $p_{\pi} (a | s)$ is trained using a linear softmax of small pattern features with weights $\pi$, this policy is fast less accurate (The purpose of this policy is to fast sample action from rollouts, more like behavior policy).
The RL policy network $p_{\rho}$ is identical in structure to the SL and has its weights $\rho$ initialized to the same values, $\rho = \sigma$. Rollouts are generated by playing games between current policy network and randomly selected previous policy network. The reward function $r(s)$ is zero for all non-terminal time steps, 1 for wining and -1 for losing. The weights are updated at each time step t by stochastic gradient ascent in the direction that maximizes expected outcome $Z_t = r(s_T)$. (REINFORCE updates with no discounting for each time step)
The value function $v^{p_\rho} \approx v_{\theta}$ is trained by minimizing the approximate error $\| z - v_{\theta} (s)\|_{D_n}^2$

The naive approach of predicting game outcomes from data consisting
of complete games leads to overfitting. The problem is that
successive positions are strongly correlated, differing by just one stone,
but the regression target is shared for the entire game. When trained
on the KGS data set in this way, the value network memorized the
game outcomes rather than generalizing to new positions, achieving a
minimum MSE of 0.37 on the test set, compared to 0.19 on the training
set. To mitigate this problem, we generated a new self-play data set
consisting of 30 million distinct positions, each sampled from a separate
game. Each game was played between the RL policy network and
itself until the game terminated.

At each simulation step (descending the tree in complete games without backup), starting from root state (initial state). At each time step t of each simulation, an action $a_t$ is selected from state $s_t$ with maximum action value Q plus a bounus term to encourage exploration.